In [1]:
#!conda install --yes -n cropmodel -c conda-forge cppyy

In [2]:
import cppyy
from ctypes import c_double, c_long, c_int

In [3]:
cppyy.include('../server/model/TGRAINS.h')
cppyy.load_library('../server/model/libTGRAINS.so')

In [4]:
from cppyy.gbl.std import vector

cropAreas = vector['double'](11)
livestockAreas = vector['double'](6)
myUniqueLandscapeID = c_int(101)
maxCropArea = c_double(0)
error = c_int(0)

In [5]:
cppyy.gbl.initialise(101, maxCropArea, cropAreas, livestockAreas, error)
cropAreas.size()

11

In [6]:
#void run(
#    vector<double> cropAreas
#    vector<double> livestockAreas
#    double& greenhouseGasEmissions
#    double& nLeach
#    vector<double>& pesticideImpacts
#    double& profit
#    double& production
#    vector<double>& nutritionaldelivery
#    vector<double>& healthRiskFactors
#    int& errorFlag);

# cropAreas defined above
# livestockAreas defined above
# error (flag) defined above
greenhouseGasEmissions = c_double(0)
nLeach                 = c_double(0)
pesticideImpacts       = vector['double'](range(5))
profit                 = c_double(0)
production             = c_double(0)
nutritionaldelivery    = vector['double']()
healthRiskFactors      = vector['double']()

In [7]:
maxCropArea

c_double(363766.22727000003)

In [8]:
for i in range(0,cropAreas.size()):
    print(cppyy.gbl.getCropString(i), '\t', cropAreas[i])

FRUIT_SOFT 	 1673.3372840000002
ALL_PEAS_BEANS 	 13387.184789395684
ALL_OATS 	 6037.01735577993
SUGARBEET 	 23171.312951484386
ALL_BARLEY 	 49777.655426557394
MAIZE 	 9796.96465172778
ALL_WHEAT 	 61877.10372481784
ALL_VEGETABLES 	 21687.045396768448
OSR_WINTER 	 32646.197396086704
ALL_POTATO 	 12936.45407116927
FRUIT_TOP 	 1909.0199859999998


In [21]:
[cppyy.gbl.getFoodGroupString(i) for i in range(9)]

['WHOLE_GRAIN',
 'STARCHY_VEG',
 'DAIRY',
 'ANIMAL_PROTEIN',
 'PLANT_PROTEIN',
 'FAT',
 'SUGAR',
 'VEGETABLE',
 'FRUIT']

In [10]:
nutritionaldelivery.size()

0

In [9]:
cppyy.gbl.getLandscapeString(101)

'East Anglia Study Area'

In [10]:
totalCropArea = 0
for d in cropAreas:
    totalCropArea = totalCropArea + d
totalCropArea

234899.29303378746

In [11]:
cropProps = vector['double']((0.1, 0.05, 0.1, 0.075, 0.125, 0.07, 0.06, 0.05, 0.05, 0.025, 0.05))
for i in range(cropAreas.size()):
    cropAreas[i] = totalCropArea * cropProps.at(i)

In [12]:
import time
start = time.time()

In [13]:
print([i for i in cropAreas])

[23489.929303378747, 11744.964651689374, 23489.929303378747, 17617.44697753406, 29362.411629223432, 16442.950512365125, 14093.957582027248, 11744.964651689374, 11744.964651689374, 5872.482325844687, 11744.964651689374]


In [14]:
# Run model
cppyy.gbl.run(
    cropAreas,
    livestockAreas,
    greenhouseGasEmissions,
    nLeach,
    pesticideImpacts,
    profit,
    production,
    nutritionaldelivery,
    healthRiskFactors,
    error
)

In [15]:
print(cropAreas.size())

11


In [16]:
end = time.time()
print("Run time = {}".format(end - start))

Run time = 1.5147101879119873


In [17]:
print(greenhouseGasEmissions)
print(nLeach)
print(profit)
print(production)

c_double(86071678.23931974)
c_double(4367915.49418526)
c_double(34624269791.59537)
c_double(395742786.8747552)


In [18]:
print([i for i in pesticideImpacts])
print([i for i in nutritionaldelivery])
print([i for i in healthRiskFactors])

[224575374493.26514, 797802655761.8436, 1207707065487.451, 1247940700597.6157, 2610830697789.0137]
[0.7772767435350761, 0.05792266460497663, 5.0676324787561944e-05, 0.0006570028529219565, 0.007018366359441477, 0.09151213699358311, 0.02109475992674452, 0.043814898221882975, 0.0006527511805857025]
[]
